In [185]:
import urllib.request as url
import pandas as pd
from geopy.distance import vincenty
import webbrowser
import json
from geopy.geocoders import Nominatim
geolocator = Nominatim()
LAT = 40.7829
LON = 73.9654
global source
source = (LAT,LON)

In [186]:
table_data = pd.read_csv('open_table_data_final.csv',encoding = "ISO-8859-1")
len(table_data)


595

In [187]:
table_data[:5]["lat"]


0    40.745962
1    40.745394
2    40.751123
3    40.748476
4    40.748698
Name: lat, dtype: float64

In [188]:
table_data[:5]["lng"]

0   -73.988622
1   -73.989588
2   -74.002069
3   -74.003339
4   -74.003228
Name: lng, dtype: float64

In [189]:
def calculateDistance(table_data,source):
    distance = []
    #print(source)
    #table_data1 = pd.DataFrame()
    for i in range(len(table_data)):
        lat_dest = table_data.at[i,"lat"]
        lng_dest = table_data.at[i,"lng"]
        dest = (lat_dest,lng_dest)
        distance.append(vincenty(source, dest).miles)
    table_data = table_data.assign(dist = distance)
    print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    #return table_data


In [190]:
#using geopy displacement
def getLatAndLng(place):
    location = geolocator.geocode(place)
    print((location.latitude, location.longitude))
    return [location.latitude,location.longitude]

In [191]:
#using osrm matrix distance
def calculateDistanceOSRM(table_data,source):
    distance = []
    #data = json.load(url.urlopen("http://router.project-osrm.org/table/v1/drive/-73.987330,40.748711;-74.0134,40.7127?sources=1"))
    for i in range(len(table_data)):
        lat_dest = table_data.at[i,"lat"]
        lng_dest = table_data.at[i,"lng"]
        json_data = url.urlopen("http://router.project-osrm.org/table/v1/drive/"+str(source[1])+","+str(source[0])+";"+str(lng_dest)+","+str(lat_dest)+"?sources=1")
        data = json.load(json_data)
        calc_distance = round((data['durations'][0][0])*0.000621371,4)
        distance.append(calc_distance)
    table_data = table_data.assign(dist = distance)
    print(table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5])
    return table_data

In [198]:
def searchForRestaurant(query,table_data):
    if("near me" in query or "nearme" in query):
        LAT = 40.7829
        LON = 73.9654
        source = (LAT,LON)
    else:
        place = query.split("near")
        loc = getLatAndLng(place)
        LAT = loc[0]
        LON = loc[1]
        source = (LAT,LON)
        table_data_dist = pd.DataFrame()
        table_data_dist = calculateDistance(table_data,source)
        #calculateDistanceOSRM(table_data,source)
        print(type(table_data_dist))
        return table_data_dist

In [199]:
table_data_dist = pd.DataFrame()
print("please enter your query to search")
table_data_dist = searchForRestaurant(input(),table_data)
print(type(table_data_dist))
table_data_dist.sort_values(by=['price','dist'], ascending=[True,True])[1:5]

please enter your query to search
near jersey city
(40.7281575, -74.0776417)
         id                             name            address      city  \
220   83113                   Kana Tapas Bar      324 Spring St  New York   
223  126202  American Cut- The Morrison Room  363 Greenwich St.  New York   
219   85597                              Pao         322 Spring  New York   
102   63811                        Warren 77   77 Warren Street  New York   

    state                       area  postal_code country        phone  \
220    NY  New York / Tri-State Area        10013      US   2123438180   
223    NY  New York / Tri-State Area        10013      US   2122264736   
219    NY  New York / Tri-State Area        10013      US   2123345464   
102    NY  New York / Tri-State Area        10007      US  2122278994x   

           lat        lng  price  \
220  40.725834 -74.009418      2   
223  40.719304 -74.010204      2   
219  40.725961 -74.009179      2   
102  40.715151 -74.010

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [ ]:
#calculateDistance()
#table_data_dist
#table_data[:5]
#table_data.sort_values(by=['price','dist'], ascending=[True,True])[1:5]